# 07 - Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [37]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [38]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess.csv'

# B. Jeux de données

### a) Metrics

In [39]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (3510431, 14)


In [40]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [41]:
metrics_df['timestamp'].min()

'2022-04-15 05:55:06.678000+00:00'

In [42]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

1242037


In [43]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,IDLE,36192,100000
1,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,IDLE,36192,100000
2,PLC,2022-04-15 06:06:56.418000+00:00,INFO,330,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,IDLE,36192,100000


### b) Jobs

In [44]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (16295, 42)


In [45]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [46]:
jobs_df['started_at'].min()

'2021-06-18 09:22:46.866000+00:00'

In [47]:
jobs_df.head(3)

,started_at,ended_at,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,jobId,total_copies_requested,LED,...,leftMargin_remoteScannerRegistration,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies,varnishConsumptionVarnish_3d_event
0,2021-06-18 09:22:46.866000+00:00,2021-06-18 09:22:46.866000+00:00,520,740,0,0,0.0,1624008166,3,10,...,0,1500,1500,1500,16,16,1,UNDEFINED,0,0.000000
1,2021-06-18 10:08:13.716000+00:00,2021-06-18 10:10:22.257000+00:00,740,520,0,0,0.0,1624010893,100,30,...,0,1500,1500,1500,16,10,1,ERROR,3,1.440239
2,2021-06-18 10:11:52.165000+00:00,2021-06-18 10:18:20.294000+00:00,740,520,0,0,0.0,1624011111,100,30,...,0,1500,1500,1500,16,10,1,CANCELED,70,33.607494


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [48]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.shape

(10516, 42)

## b) Réduction de metrics

### Par source d'évènement

In [49]:
metrics_df['source_events'].unique()

array(['iFoil', 'PLC', nan, 'Kernel', 'RCB n°1', 'RCB n°2', 'ICB n°1',
       'RCB n°3', 'ICB n°5', 'ICB n°7', 'ICB n°8', 'Pilot', 'ICB n°4',
       'ICB n°2', 'ICB n°6'], dtype=object)

In [50]:
# on se concentre sur les données de metrics qui ont une source d'évènement
metrics_by_source = metrics_df[metrics_df['source_events'].notna()]
metrics_by_source.shape

(94166, 14)

In [51]:
# on vérifie la distribution par criticité
metrics_by_source['criticality_events'].value_counts()

INFO       68501
WARNING    12848
ERROR      12817
Name: criticality_events, dtype: int64

### Par identifiant d'évènement

In [52]:
# lignes avec des évènements identifiant une maintenance
id_to_drop_1 = [391, 330, 377, 407, 332, 331, 313, 333, 376, 372, 344, 343, 371, 358, 334, 311, 472, 0, 408, 406, 350, 2, 352, 346]
# lignes avec des évènements identifiant une intervention humaine
id_to_drop_2 = [352, 324, 381, 440, 385, 405, 447, 388, 320, 417, 444, 329, 315, 384, 345, 349, 466, 419]
# fusion des listes d'identifiants à supprimer
id_to_drop = id_to_drop_1 + id_to_drop_2
# on supprime les lignes avec des évènements identifiant une maintenance
metrics_by_identification = metrics_by_source[~metrics_by_source['identification_events'].isin(id_to_drop)]
metrics_by_identification.shape

(8062, 14)

In [53]:
# on vérifie la distribution par criticité
metrics_by_identification['criticality_events'].value_counts()

ERROR      5933
INFO       1530
WARNING     599
Name: criticality_events, dtype: int64

In [94]:
# Pour ne pas avoir un jeu de données trop petit on ne supprime que les 4 premiers idnetifiants les plus représentés mais les moins importants
# metrics_by_identification_1 = metrics_by_source[~metrics_by_source['identification_events'].isin([331,330,334,332])]
# metrics_by_identification_1['criticality_events'].value_counts()

In [55]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
#len(metrics_by_criticity.groupby('timestamp').groups.keys())

# D. Création du datatset pour le pre-processing

In [56]:
metrics_reduced = metrics_by_identification.copy()

## a) Comparaison des dates entre dataset

In [57]:
# Vérifier si des dates communes existent
dates_communes_exist = jobs_reduced['started_at'].isin(metrics_reduced['timestamp']).any()

# Afficher le résultat
if dates_communes_exist:
    # Filtrer les dates communes
    dates_communes = jobs_df['started_at'][jobs_df['started_at'].isin(metrics_df['timestamp'])]
    
    # Compter le nombre de dates communes
    nombre_dates_communes = len(dates_communes)
    
    print(f"Des dates communes existent entre les ensembles de données: {nombre_dates_communes}")
else:
    print("Aucune date commune n'a été trouvée entre les ensembles de données.")


Aucune date commune n'a été trouvée entre les ensembles de données.


### Nombre de lignes par mois pour chaque dataset

In [58]:
def compare_datetime_series_shapes(s1, s2):
    s1_serie = {}
    s2_serie = {}
    s1 = pd.to_datetime(s1)
    s2 = pd.to_datetime(s2)
    for i in range(1,13):
        s1_month = s1[s1.dt.month == i]
        s2_month = s2[s2.dt.month == i]
        print(i, s1_month.shape, s2_month.shape)
        s1_serie[i] = s1_month.index
        s2_serie[i] = s2_month.index
    return s1_serie, s2_serie

In [59]:
# on liste les index des lignes par mois
jobs_indexes_by_month, metrics_indexes_by_month = compare_datetime_series_shapes(jobs_reduced['started_at'], metrics_reduced['timestamp'])

1 (0,) (0,)
2 (0,) (0,)
3 (0,) (0,)
4 (728,) (586,)
5 (1378,) (1128,)
6 (1615,) (1459,)
7 (1290,) (1354,)
8 (973,) (658,)
9 (1276,) (551,)
10 (1371,) (814,)
11 (1239,) (922,)
12 (646,) (590,)


### Analyse du nombre de données sur un mois

In [60]:
# month = 5

# Selection des lignes de jobs en Mai
# jobs_df['started_at'] = pd.to_datetime(jobs_df['started_at'])
# jobs_batch = jobs_df[ (jobs_df['started_at'].dt.month == month)]
# jobs_batch = jobs_df.loc[jobs_df.index.isin(jobs_indexes_by_month.get(month))]
# jobs_batch.shape

# Selection des lignes de metrics en Mai
# metrics_df['timestamp'] = pd.to_datetime(metrics_df['timestamp'])
# metrics_batch = metrics_df[ (metrics_df['timestamp'].dt.month == month)]
# metrics_batch = metrics_df.loc[metrics_df.index.isin(metrics_indexes_by_month.get(month))]
# metrics_batch.shape

## b) Concaténation des datasets

### Pour chaque ligne de metrics quel est le jobId correspondant

In [62]:
# on prépare la concaténation des données de jobs et metrics du mois de juin

month = 0

jobs_reduced['started_at'] = pd.to_datetime(jobs_reduced['started_at'])
metrics_reduced['timestamp'] = pd.to_datetime(metrics_reduced['timestamp'])

if month == 0 :
    # DataFrame 1 avec les intervalles de temps
    dataframe1 = jobs_reduced.copy()
    # DataFrame 2 avec les valeurs de date
    dataframe2 = metrics_reduced.copy()
else :
     # DataFrame 1 avec les intervalles de temps
    dataframe1 = jobs_reduced[(jobs_reduced['started_at'].dt.month == month)].copy()
    # DataFrame 2 avec les valeurs de date
    dataframe2 = metrics_reduced[(metrics_reduced['timestamp'].dt.month == month)].copy()
   

# Conversion des colonnes en format de date
dataframe1['started_at'] = pd.to_datetime(dataframe1['started_at'])
dataframe1['ended_at'] = pd.to_datetime(dataframe1['ended_at'])
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])

# Création de la nouvelle colonne dans dataframe2 pour l'association
dataframe2['jobId'] = 0

# Parcours des lignes de metrics
for index2, row2 in dataframe2.iterrows():
    metrics_timestamp = row2['timestamp']
    # Vérification pour chaque ligne de job
    for index1, row1 in dataframe1.iterrows():
        if row2['timestamp'] >= row1['started_at'] and row2['timestamp']  <= row1['ended_at']:
            dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
            break

C:\Users\Audrey\AppData\Local\Temp\ipykernel_9448\1114395518.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_reduced['started_at'] = pd.to_datetime(jobs_reduced['started_at'])


In [84]:
# on vérifie la distribution par criticité
dataframe2['criticality_events'].value_counts()

ERROR      5933
INFO       1530
WARNING     599
Name: criticality_events, dtype: int64

In [85]:
# on concat les données de metrics et jobs
concat_df = pd.concat([dataframe1, dataframe2]).reset_index(drop=True)
concat_df.shape

(18578, 56)

In [86]:
# on vérifie la distribution par criticité
concat_df['criticality_events'].value_counts()

ERROR      5933
INFO       1530
WARNING     599
Name: criticality_events, dtype: int64

### Suppression des lignes de metrics sans jobId

In [87]:
# on ne conserve que les lignes qui ont un jobId
concat_df = concat_df[concat_df.jobId != 0]
concat_df.shape

(12119, 56)

### Chaque ligne de metrics récupère les valeurs du jobId correspondant

In [88]:
# on remplace les valeurs maquantes des lignes metrics par celles de jobs avec le jobId correspondant
for job_id in concat_df['jobId'].unique():
    if concat_df['jobId'].value_counts().get(job_id, 0) > 1:
        concat_df.loc[concat_df['jobId'] == job_id] = concat_df.loc[concat_df['jobId'] == job_id].fillna(method='ffill')

In [89]:
# on vérifie la distribution par criticité
concat_df['criticality_events'].value_counts()

ERROR      1572
INFO         18
WARNING      13
Name: criticality_events, dtype: int64

### Suppression des lignes de jobs

In [91]:
# chaque ligne sans valeur timestamp est une ligne de jobs
concat_df_1 = concat_df[~concat_df['timestamp'].isnull()]
concat_df_1.shape

(1603, 56)

In [92]:
# on vérifie la distribution par criticité
concat_df_1['criticality_events'].value_counts()

ERROR      1572
INFO         18
WARNING      13
Name: criticality_events, dtype: int64

## d) Output

In [93]:
concat_df_1.to_csv(path_or_buf=save_csv)